### 최종

In [4]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# PNU별 매출액
sql = '''
select substring (bd_mgt_sn, 0, 20) as pnu, pred_slng_amt as sales, ftc_cate2_cd, emd_cd||'00'
  from m1.kt_bldg_sales
 where base_ym = '202209'
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_kt_bldg_sales = pd.DataFrame(rows)
df_kt_bldg_sales.columns = ['pnu','sales','industry_cd','emd_cd']

# 법정동별 소득 
sql = '''
select cur_lgl_dong_cd||'00' as emd_nm, avg(avg_mon_income) as income_avg
  from m1.kcb_stat_cm
 where bs_yr_mon = '202209'
 group by cur_lgl_dong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_kcb_stat_cm = pd.DataFrame(rows)
df_kcb_stat_cm.columns = ['emd_cd','income_avg']


# 데이터 프레임 병합
df = pd.merge(df_kt_bldg_sales, df_kcb_stat_cm, on='emd_cd', how='left')

# 소득이 0명, null인 지역 제외
df = df[~df['income_avg'].isnull()]

# 매출액 / 소득
df['income_per_sales'] = (df['sales'] / df['income_avg']) *1000 # 원(WON) 단위로 변경
df = df[['pnu','emd_cd','income_per_sales','industry_cd']]

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'


### 2. DB INSERT
import datetime

table = 'm2.datadam_income_per_sales'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-11-08 17:13:56.064135  :  0
2022-11-08 17:13:58.490413  :  10000
2022-11-08 17:14:00.392038  :  20000
2022-11-08 17:14:03.470623  :  30000
2022-11-08 17:14:06.158737  :  40000
2022-11-08 17:14:08.003310  :  50000
2022-11-08 17:14:10.347540  :  60000
2022-11-08 17:14:13.206238  :  70000
2022-11-08 17:14:15.420212  :  80000
2022-11-08 17:14:17.949901  :  90000
2022-11-08 17:14:21.877675  :  100000
2022-11-08 17:14:24.239403  :  110000
2022-11-08 17:14:26.477305  :  120000
2022-11-08 17:14:28.894096  :  130000
2022-11-08 17:14:31.448748  :  140000
2022-11-08 17:14:35.147931  :  150000
2022-11-08 17:14:37.237834  :  160000
2022-11-08 17:14:40.129492  :  170000
2022-11-08 17:14:41.979112  :  180000
2022-11-08 17:14:44.193458  :  190000
2022-11-08 17:14:46.113131  :  200000
2022-11-08 17:14:47.961187  :  210000
2022-11-08 17:14:50.459392  :  220000
2022-11-08 17:14:54.188714  :  230000
2022-11-08 17:14:56.319145  :  240000
2022-11-08 17:14:59.799146  :  250000
2022-11-08 17:15:02.814620

In [3]:
df

,pnu,emd_cd,income_per_sales,industry_cd,create_at,update_at,work_user
0,3114010100108180007,3114010100,6166526.3158,A15,2022-11-08,2022-11-08,du.Park
1,3114010100108180007,3114010100,3687027.7172,B18,2022-11-08,2022-11-08,du.Park
2,3114010100108180007,3114010100,311429.4612,B09,2022-11-08,2022-11-08,du.Park
3,3114010100108180007,3114010100,2676839.6138,A05,2022-11-08,2022-11-08,du.Park
4,3114010100108180007,3114010100,19699648.0847,A01,2022-11-08,2022-11-08,du.Park
...,...,...,...,...,...,...,...
58555804,5013032026119420001,5013032000,990386.7833,A06,2022-11-08,2022-11-08,du.Park
58555805,5013032026119420001,5013032000,3937185.2932,C06,2022-11-08,2022-11-08,du.Park
58555806,5013032026119420001,5013032000,1541254.9401,C02,2022-11-08,2022-11-08,du.Park
58555807,5013032026119420001,5013032000,1961488.5002,B03,2022-11-08,2022-11-08,du.Park


In [2]:
df_kt_bldg_sales

,pnu,sales,industry_cd,emd_cd
0,3114010100102820002,5740478,B14,3114010100
1,3114010100102820002,100000000,B21,3114010100
2,3114010100102820002,1323405,B20,3114010100
3,3114010100102820002,12688502,A07,3114010100
4,3114010100102820002,8821770,A02,3114010100
...,...,...,...,...
58555804,5013032026117900000,14151894,C01,5013032000
58555805,5013032026117900000,8786831,B10,5013032000
58555806,5013032026117900000,2477928,A05,5013032000
58555807,5013032026117900000,26818739,A10,5013032000


In [3]:
df_kcb_stat_cm

,emd_cd,income_avg
0,1111013000,3033
1,1111016800,3121
2,1111017500,3045
3,1111018600,3784
4,1114010700,4540
...,...,...
4979,4136011000,4296
4980,4613014500,4031
4981,1114017200,4416
4982,3020014800,2466


In [11]:
df

,pnu,emd_cd,income_per_sales,industry_cd,create_at,update_at,work_user
0,3114010100102820002,3114010100,1787753.9707,B14,2022-11-07,2022-11-07,du.Park
1,3114010100102820002,3114010100,31142946.1227,B21,2022-11-07,2022-11-07,du.Park
2,3114010100102820002,3114010100,412147.3061,B20,2022-11-07,2022-11-07,du.Park
3,3114010100102820002,3114010100,3951573.3416,A07,2022-11-07,2022-11-07,du.Park
4,3114010100102820002,3114010100,2747359.0782,A02,2022-11-07,2022-11-07,du.Park
...,...,...,...,...,...,...,...
58555804,5013032026117900000,5013032000,4584351.7979,C01,2022-11-07,2022-11-07,du.Park
58555805,5013032026117900000,5013032000,2846398.1212,B10,2022-11-07,2022-11-07,du.Park
58555806,5013032026117900000,5013032000,802697.7648,A05,2022-11-07,2022-11-07,du.Park
58555807,5013032026117900000,5013032000,8687638.1600,A10,2022-11-07,2022-11-07,du.Park


In [13]:
df['income_per_sales'].median()

2462456.7510548523